# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
import pandas as pd
import os
import requests

# URL to the CSV file
url1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv"
url2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.names"
target_dir = "../../05_src/data/fires/"

os.makedirs(target_dir, exist_ok=True)

# Step 4: Write the content to a file
def download_file(url, filename):
    response = requests.get(url)
    file_path = os.path.join(target_dir, filename)
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved at: {file_path}")

download_file(url1, "forestfires.csv")
download_file(url2, "forestfires.names")

File downloaded and saved at: ../../05_src/data/fires/forestfires.csv
File downloaded and saved at: ../../05_src/data/fires/forestfires.names


In [2]:
# Define file path
csv_path = os.path.join(target_dir, "forestfires.csv")

# Read the CSV into a DataFrame
forest_fire_data = pd.read_csv(csv_path)

# Display the first few rows of the DataFrame
print(forest_fire_data.head())

   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0


In [3]:
# Define the path to the .names file
names_file_path = "../../05_src/data/fires/forestfires.names"

# Open and read the .names file with a different encoding
with open(names_file_path, "r", encoding='ISO-8859-1') as file:
    names_content = file.read()

# Display the content of the .names file
print(names_content)


Citation Request:
  This dataset is public available for research. The details are described in [Cortez and Morais, 2007]. 
  Please include this citation if you plan to use this database:

  P. Cortez and A. Morais. A Data Mining Approach to Predict Forest Fires using Meteorological Data. 
  In J. Neves, M. F. Santos and J. Machado Eds., New Trends in Artificial Intelligence, 
  Proceedings of the 13th EPIA 2007 - Portuguese Conference on Artificial Intelligence, December, 
  Guimaraes, Portugal, pp. 512-523, 2007. APPIA, ISBN-13 978-989-95618-0-9. 
  Available at: http://www.dsi.uminho.pt/~pcortez/fires.pdf

1. Title: Forest Fires

2. Sources
   Created by: Paulo Cortez and Aníbal Morais (Univ. Minho) @ 2007
   
3. Past Usage:

   P. Cortez and A. Morais. A Data Mining Approach to Predict Forest Fires using Meteorological Data.
   In Proceedings of the 13th EPIA 2007 - Portuguese Conference on Artificial Intelligence, 
   December, 2007. (http://www.dsi.uminho.pt/~pcortez/fires.pdf)


# Get X and Y

Create the features data frame and target data.

In [4]:
from sklearn.model_selection import train_test_split

# Define the target variable (assuming 'area' is the target)
Y = forest_fire_data['area']

# Create the features DataFrame by dropping the target column and any other non-feature columns

X = forest_fire_data.drop(columns=['area'])  # Adjust columns as necessary

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (413, 12)
X_test shape: (104, 12)
Y_train shape: (413,)
Y_test shape: (104,)


In [5]:
X_train.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain
329,4,3,sep,sat,92.2,102.3,751.5,8.4,23.5,27,4.0,0.0
173,4,4,sep,mon,90.9,126.5,686.5,7.0,17.7,39,2.2,0.0
272,2,5,aug,tue,92.1,152.6,658.2,14.3,20.2,47,4.0,0.0
497,3,4,aug,tue,96.1,181.1,671.2,14.3,32.3,27,2.2,0.0
182,5,4,feb,sun,86.8,15.6,48.3,3.9,12.4,53,2.2,0.0


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np


In [7]:
# Define numerical and categorical features
numerical_features = ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']
categorical_features = ['month', 'day', 'X', 'Y']

preproc1 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', KNNImputer(n_neighbors=5, weights='distance')),  # Step 1: Impute with KNN
            ('scaler', RobustScaler()),  # Step 2: Scale numeric data
        ]), numerical_features),  
        ('cat', Pipeline([             
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Step 1: Impute with the most frequent value
            ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Step 2: One-hot encoding for categorical features
        ]), categorical_features)
    ]
)

In [8]:
preproc1

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(weights='distance')),
                                                 ('scaler', RobustScaler())]),
                                 ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH',
                                  'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['month', 'day', 'X', 'Y'])])

### Preproc 2

Create preproc2 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [9]:
# Define a function for the logarithmic transformation
def log_transform(X):
    return np.log1p(np.maximum(X, 1e-5))  # Use np.log1p to handle zero values gracefully

# Create the second ColumnTransformer (preproc2)
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', KNNImputer(n_neighbors=5)),         # Step 1: Impute with KNN
            ('scaler', RobustScaler()),                   # Step 2: Scale numeric data
            ('log', FunctionTransformer(log_transform))     # Step 3: Apply log transformation
        ]), numerical_features),  
        ('cat', Pipeline([             
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Step 1: Impute with the most frequent value
            ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Step 2: One-hot encoding for categorical features
        ]), categorical_features)      # One-hot encoding for categorical features
    ]
)


In [10]:
preproc2

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', KNNImputer()),
                                                 ('scaler', RobustScaler()),
                                                 ('log',
                                                  FunctionTransformer(func=<function log_transform at 0x00000244832C5A60>))]),
                                 ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH',
                                  'wind', 'rain']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['month', 'day', 'X', 'Y'])])

In [11]:
X1 = preproc1.fit_transform(X_train)
X1_test_transformed = preproc1.transform(X_test)
X2 = preproc2.fit_transform(X_train)
X2_test_transformed = preproc2.transform(X_test)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [13]:
# Pipeline A: preproc1 + Linear Regression (baseline)
pipeline_A = Pipeline(steps=[
    ('preprocessing', preproc1),  # Preprocessing using preproc1
    ('regressor', LinearRegression())  # Baseline regressor
])


In [14]:
pipeline_A

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['FFMC', 'DMC', 'DC', 'ISI',
                                                   'temp', 'RH', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['month', 'day', 'X',
                                                   'Y'])])),
                ('regressor', LinearRegression())])

In [15]:
# Pipeline B: preproc2 + Linear Regression (baseline)

pipeline_B = Pipeline(steps=[
    ('preprocessing', preproc2),  # Preprocessing using preproc2
    ('regressor', LinearRegression())  # Baseline regressor
])


In [16]:
pipeline_B

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x00000244832C5A60>))]),
                                                  ['FFMC', 'DMC', 'DC', 'ISI',
                                                   'temp', 'RH', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['month', 'day', 'X',
                                                   'Y'])])),
                ('regressor', LinearRegression())])

In [17]:
# Pipeline C = preproc1 + advanced model
# Pipeline C: preproc1 + Random Forest Regressor (advanced model)
pipeline_C = Pipeline(steps=[
    ('preprocessing', preproc1),  # Preprocessing using preproc1
    ('regressor', RandomForestRegressor())  # Advanced regressor
])


In [18]:
pipeline_C

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['FFMC', 'DMC', 'DC', 'ISI',
                                                   'temp', 'RH', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['month', 'day', 'X',
                                                   'Y'])])),
                ('regressor', RandomForestRegressor())])

In [19]:
# Pipeline D = preproc2 + advanced model
# Pipeline D: preproc2 + Random Forest Regressor (advanced model)
pipeline_D = Pipeline(steps=[
    ('preprocessing', preproc2),  # Preprocessing using preproc2
    ('regressor', RandomForestRegressor())  # Advanced regressor
])

    

In [20]:
pipeline_D

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer()),
                                                                  ('scaler',
                                                                   RobustScaler()),
                                                                  ('log',
                                                                   FunctionTransformer(func=<function log_transform at 0x00000244832C5A60>))]),
                                                  ['FFMC', 'DMC', 'DC', 'ISI',
                                                   'temp', 'RH', 'wind',
                                                   'rain']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['month', 'day', 'X',
                                                   'Y'])])),
                ('regressor', RandomForestRegressor())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [21]:
# Pipeline A: preproc1 + Linear Regression
param_grid_A = {
    'regressor__fit_intercept': [True, False]
}

param_grid_B = {
    'regressor__fit_intercept': [True, False]  # Same as A for consistency
}

# Pipeline C: Tune the 'max_depth' and 'n_estimators' of Random Forest
param_grid_C = {
    'regressor__max_depth': [None, 10, 20, 30],  # None means nodes are expanded until all leaves are pure
    'regressor__n_estimators': [50, 100, 200]    # Number of trees in the forest
}

# Pipeline D: Tune the 'max_depth' and 'min_samples_split' of Random Forest
param_grid_D = {
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__min_samples_split': [2, 5, 10]  # Minimum number of samples required to split an internal node
}


In [22]:
# Run grid search on pipeline_A
grid_cv_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_cv_A.fit(X_train, Y_train)

# Access the cross-validation results
res_A = grid_cv_A.cv_results_

# Display keys in cv_results_ to see what information is available
print(res_A.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_regressor__fit_intercept', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [23]:

res_A = pd.DataFrame(res_A)
res_A.columns

res_A[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_regressor__fit_intercept', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score']].sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.046875,0.002599,0.011768,0.000747,True,{'regressor__fit_intercept': True},-1775.277125,-360.867080,-1420.685025,-7339.943511,-714.156463,-2322.185841,2558.211667,1
1,0.041488,0.003711,0.011570,0.003920,False,{'regressor__fit_intercept': False},-1776.187578,-360.856671,-1420.901681,-7341.819587,-713.331262,-2322.619356,2558.998805,2


In [24]:
grid_search_A = GridSearchCV(pipeline_A, param_grid_A, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_A.fit(forest_fire_data.drop('area', axis=1), forest_fire_data['area'])
best_params_A = grid_search_A.best_params_
best_mse_A = -grid_search_A.best_score_
print(f"Pipeline A Best parameters: {best_params_A}")
print(f"Pipeline A MSE (cross-validated): {best_mse_A}")

Pipeline A Best parameters: {'regressor__fit_intercept': False}
Pipeline A MSE (cross-validated): 4558.669073766401


In [25]:
grid_search_B = GridSearchCV(pipeline_B, param_grid_B, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_B.fit(forest_fire_data.drop('area', axis=1), forest_fire_data['area'])
best_params_B = grid_search_B.best_params_
best_mse_B = -grid_search_B.best_score_
print(f"Pipeline B Best parameters: {best_params_B}")
print(f"Pipeline B MSE (cross-validated): {best_mse_B}")

Pipeline B Best parameters: {'regressor__fit_intercept': True}
Pipeline B MSE (cross-validated): 4610.781650224055


In [26]:
grid_search_C = GridSearchCV(pipeline_C, param_grid_C, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_C.fit(forest_fire_data.drop('area', axis=1), forest_fire_data['area'])
best_params_C = grid_search_C.best_params_
best_mse_C = -grid_search_C.best_score_
print(f"Pipeline C Best parameters: {best_params_C}")
print(f"Pipeline C MSE (cross-validated): {best_mse_C}")

Pipeline C Best parameters: {'regressor__max_depth': 30, 'regressor__n_estimators': 200}
Pipeline C MSE (cross-validated): 4979.299173803891


In [27]:
grid_search_D = GridSearchCV(pipeline_D, param_grid_D, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_D.fit(forest_fire_data.drop('area', axis=1), forest_fire_data['area'])
best_params_D = grid_search_D.best_params_
best_mse_D = -grid_search_D.best_score_
print(f"Pipeline D Best parameters: {best_params_D}")
print(f"Pipeline D MSE (cross-validated): {best_mse_D}")

Pipeline D Best parameters: {'regressor__max_depth': 10, 'regressor__min_samples_split': 5}
Pipeline D MSE (cross-validated): 4888.315473417036


# Evaluate

+ Which model has the best performance?

Pipeline A has the best performance as it has the lowest MSE of 4558.67. This suggests that its configuration effectively captures the relationships in the data while minimizing prediction errors.

# Export

+ Save the best performing model to a pickle file.

In [28]:
import pickle

# Assuming grid_cv_A is your GridSearchCV object for Pipeline A
grid_cv_A.fit(forest_fire_data.drop('area', axis=1), forest_fire_data['area'])  # Fit the model

# Retrieve the best model from GridSearchCV
best_model_A = grid_cv_A.best_estimator_

# Save the best model to a pickle file
with open('best_model_A.pkl', 'wb') as file:
    pickle.dump(best_model_A, file)

print("Best performing model saved to best_model_A.pkl")


Best performing model saved to best_model_A.pkl


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [31]:
import shap

In [32]:
import pickle

# Load the best model
with open('best_model_A.pkl', 'rb') as file:
    best_model_A = pickle.load(file)


In [34]:
# Initialize the SHAP explainer
explainer = shap.Explainer(pipeline_A.named_steps['regressor'], X_train)

# Calculate SHAP values for the test set
shap_values = explainer(X_test)

# Select a specific observation from X_test, e.g., the first observation
observation = X_test.iloc[0]  # You can change the index to select different observations

# Get the SHAP values for the selected observation
observation_index = 0  # Change this index based on which observation you want to analyze
observation_shap_values = shap_values[observation_index]

# Plot the SHAP values for the specific observation
shap.initjs()  # Initialize JS visualization
shap.force_plot(explainer.expected_value, observation_shap_values.values, observation)


TypeError: The passed model is not callable and cannot be analyzed directly with the given masker! Model: LinearRegression()

Couldn't figure how to do SHAP in time. Probably have to go through the documentation, sample notebook, and code to continue to get a better understanding of the theory and how the code works.

## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.